# Document Clustering by DEC(Deep Embedded Clustering)

### References

* Preprocess: https://towardsdatascience.com/topic-modeling-and-latent-dirichlet-allocation-in-python-9bf156893c24
* DEC: 
    - https://github.com/Tony607/Keras_Deep_Clustering/blob/master/Keras-DEC.ipynb
    - https://github.com/XifengGuo/DEC-keras
* overall : https://github.com/madigun697/data_analysis/blob/master/Topic_Modeling/Topic_Modeling_DEC_Drug.ipynb

## Preprocess data

In [1]:
import pandas as pd
import numpy as np
import nltk
import re

pd.set_option('display.max_colwidth', 999)
drug_data = pd.read_csv('../data/titles_condition_by_t.tsv', sep='\t', header=None)

In [2]:
drug_data.columns = ['id', 'title']
drug_data.head()

,id,title
0,4106,Analysis of efficacy
1,4107,Comparisons of postoperative CA19-9 levels on survival of ESPAC-4 with the CONOKO-01 and JASPAC-1 trials
2,4108,Pattern of disease relapse
3,4109,Grade 1–5 adverse events with gemcitabine alone and gemcitabine plus capecitabine
4,4112,Treatment with zoledronic acid


In [3]:
drug_data.title = drug_data.title.str.strip()
drug_data['title'].replace('', np.nan, inplace=True)
print(drug_data.isna().any())

id       False
title    False
dtype: bool


In [4]:
# rep = {'nbsp':'', 'table':'', 'legend':'', 'mg/dl':'', 'g/l':'', 'yrs':'year', '\n':' ', ';':'', 'kg/m2':'', 'n=':''}
rep = {'nbsp':'', 'table':'', 'legend':'', 'yrs':'year', '\n':' '}
# clean_content = [pattern.sub(lambda m: rep[re.escape(m.group(0))], x['content']) for x in train_data]
rep = dict((re.escape(k), v) for k, v in rep.items())
pattern = re.compile("|".join(rep.keys()))
drug_data.title = [pattern.sub(lambda m: rep[re.escape(m.group(0))], str(x)) for x in drug_data.title]

In [5]:
drug_data.title[:10]

0                                                                                        Analysis of efficacy
1    Comparisons of postoperative CA19-9 levels on survival of ESPAC-4 with the CONOKO-01 and JASPAC-1 trials
2                                                                                  Pattern of disease relapse
3                           Grade 1–5 adverse events with gemcitabine alone and gemcitabine plus capecitabine
4                                                                              Treatment with zoledronic acid
5                                                                                    Treatment with docetaxel
6                                Treatments ever used at relapse, at the discretion of the treating clinician
7                                            Worst adverse event  (grade)  reported over entire time on trial
8                                                        Chemotherapy delivery and trial drug discontinuation
9         

In [6]:
import gensim
from gensim.utils import simple_preprocess
from gensim.parsing.preprocessing import STOPWORDS
from gensim.parsing.preprocessing import strip_numeric
from nltk.stem import WordNetLemmatizer, SnowballStemmer
from nltk.stem.porter import *

In [7]:
np.random.seed(2018)

import nltk
nltk.download('wordnet')

[nltk_data] Downloading package wordnet to /Users/grace/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [8]:
#stopwords
stemmer = SnowballStemmer('english')
STOP_WORDS = list(gensim.parsing.preprocessing.STOPWORDS)
STOP_WORDS.extend(['table', 'legend'])

In [9]:
# stemming
# -porter stemmer
# -lancaster stemmer
# -snowball stemmer
from nltk.stem.lancaster import LancasterStemmer
from nltk.stem.snowball import SnowballStemmer

def lemmatize_stemming(text):
    return stemmer.stem(WordNetLemmatizer().lemmatize(text, pos='v'))

def preprocess(text):
    result = []
#     deacc=True removes punctuations
    for token in gensim.utils.simple_preprocess(text, deacc=True):
        if token not in STOP_WORDS and len(token)>1:
#             result.append(lemmatize_stemming(strip_numeric(token)))
            result.append(lemmatize_stemming(token))
    return result

def preprocess_token_only(text):
    result = []
#     deacc=True removes punctuations
    for token in gensim.utils.simple_preprocess(text, deacc=True):
        if token not in STOP_WORDS and len(token)>1:
#             result.append(lemmatize_stemming(strip_numeric(token)))
            result.append(token)
    return result

stemmer = LancasterStemmer()
#tokenizing
def tokenize_and_stem(text):
    tokens = [word.lower() for sent in nltk.sent_tokenize(text) for word in nltk.word_tokenize(sent)]
    #filter tokens not containing letters
    filtered = []
    for token in tokens:
        if re.search('[a-zA-Z]', token) and len(token)>2:
            filtered.append(token)
#     stems = [stemmer.stem(t, pos='v') for t in filtered]
    stems = [stemmer.stem(t) for t in filtered]
    return stems

def tokenize_only(text):
    tokens = [word.lower() for sent in nltk.sent_tokenize(text) for word in nltk.word_tokenize(sent)]
    filtered = []
    for token in tokens:
        if re.search('[a-zA-Z]', token) and len(token)>2:
            filtered.append(token)
    return filtered

In [10]:
%time drug_data['processed'] = drug_data['title'].map(preprocess)
drug_data.processed[:10]

CPU times: user 10.4 s, sys: 90.6 ms, total: 10.5 s
Wall time: 10.5 s


0                                                        [analys, eff]
1    [comparison, postop, ca, level, surv, espac, conoko, jaspac, tri]
2                                            [pattern, diseas, relaps]
3         [grad, advers, ev, gemcitabin, gemcitabin, plu, capecitabin]
4                                                [tre, zoledron, acid]
5                                                     [tre, docetaxel]
6                                    [tre, relaps, discret, tre, clin]
7                   [worst, advers, ev, grad, report, entir, tim, tri]
8                      [chemotherapy, delivery, tri, drug, discontinu]
9                                                         [advers, ev]
Name: processed, dtype: object

In [ ]:
# from sklearn.feature_extraction.text import CountVectorizer
# vect = CountVectorizer()
# %time vect.fit([' '.join(d) for d in drug_data.processed])
# %time tsne_data = vect.transform([' '.join(d) for d in drug_data.processed]).toarray()
# %time tsne_result = TSNE(learning_rate=300, init='pca').fit_transform(np.array(tsne_data))

In [11]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.preprocessing import normalize
from sklearn.manifold import TSNE

tfidf_vectorizer = TfidfVectorizer(max_df=0.8, 
                                   max_features=10000, 
                                   min_df=0.01, 
                                   stop_words='english', 
                                   use_idf=True, 
                                   lowercase=True, 
                                   tokenizer=preprocess)
#                                    tokenizer=preprocess, ngram_range=(1,2))

tfidf_vectorizer.fit(drug_data.title)
%time tfidf_data = tfidf_vectorizer.transform(drug_data.title).toarray()
print(tfidf_data.shape)

TfidfVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=0.8, max_features=10000, min_df=0.01,
        ngram_range=(1, 1), norm='l2', preprocessor=None, smooth_idf=True,
        stop_words='english', strip_accents=None, sublinear_tf=False,
        token_pattern='(?u)\\b\\w\\w+\\b',
        tokenizer=<function preprocess at 0x11f0c4e18>, use_idf=True,
        vocabulary=None)

CPU times: user 8.99 s, sys: 29.3 ms, total: 9.02 s
Wall time: 9.03 s
(27960, 147)


### L2 Normalization
 - L2 Norm은 n 차원(유클리드 공간)에서의 벡터의 크기를 계산함.

In [15]:
# %time tfidf_tsne_result = TSNE(learning_rate=300, init='pca')\
#                     .fit_transform(np.array(tfidf_data))
# tfidf_vect = normalize(tfidf_tsne_result, norm='l2')
# tfidf_tsne_result[:10]

CPU times: user 15min 50s, sys: 48.1 s, total: 16min 38s
Wall time: 16min 40s


In [18]:
import numpy as np

from keras.models import Model
from keras import backend as K
from keras import layers
from keras.layers import Input, Dense, Conv2D, MaxPooling2D, UpSampling2D, Flatten, Reshape, Conv2DTranspose
from keras.models import Model
from keras.engine.topology import Layer, InputSpec

from sklearn.cluster import KMeans
from sklearn import metrics

np.random.seed(0)

/Users/grace/workspace/keras/venv/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.
/Users/grace/workspace/keras/venv/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: compiletime version 3.5 of module 'tensorflow.python.framework.fast_tensor_util' does not match runtime version 3.6
  return f(*args, **kwds)


In [19]:
class ClusteringLayer(Layer):
    """
    Clustering layer converts input sample (feature) to soft label, i.e. a vector that represents the probability of the
    sample belonging to each cluster. The probability is calculated with student's t-distribution.

    # Example
    ```
        model.add(ClusteringLayer(n_clusters=10))
    ```
    # Arguments
        n_clusters: number of clusters.
        weights: list of Numpy array with shape `(n_clusters, n_features)` witch represents the initial cluster centers.
        alpha: degrees of freedom parameter in Student's t-distribution. Default to 1.0.
    # Input shape
        2D tensor with shape: `(n_samples, n_features)`.
    # Output shape
        2D tensor with shape: `(n_samples, n_clusters)`.
    """

    def __init__(self, n_clusters, weights=None, alpha=1.0, **kwargs):
        if 'input_shape' not in kwargs and 'input_dim' in kwargs:
            kwargs['input_shape'] = (kwargs.pop('input_dim'),)
        super(ClusteringLayer, self).__init__(**kwargs)
        self.n_clusters = n_clusters
        self.alpha = alpha
        self.initial_weights = weights
        self.input_spec = InputSpec(ndim=2)

    def build(self, input_shape):
        assert len(input_shape) == 2
        input_dim = input_shape[1]
        self.input_spec = InputSpec(dtype=K.floatx(), shape=(None, input_dim))
        self.clusters = self.add_weight((self.n_clusters, input_dim), initializer='glorot_uniform', name='clusters')
        if self.initial_weights is not None:
            self.set_weights(self.initial_weights)
            del self.initial_weights
        self.built = True

    def call(self, inputs, **kwargs):
        """ student t-distribution, as same as used in t-SNE algorithm.
         Measure the similarity between embedded point z_i and centroid µ_j.
                 q_ij = 1/(1+dist(x_i, µ_j)^2), then normalize it.
                 q_ij can be interpreted as the probability of assigning sample i to cluster j.
                 (i.e., a soft assignment)
        Arguments:
            inputs: the variable containing data, shape=(n_samples, n_features)
        Return:
            q: student's t-distribution, or soft labels for each sample. shape=(n_samples, n_clusters)
        """
        q = 1.0 / (1.0 + (K.sum(K.square(K.expand_dims(inputs, axis=1) - self.clusters), axis=2) / self.alpha))
        q **= (self.alpha + 1.0) / 2.0
        q = K.transpose(K.transpose(q) / K.sum(q, axis=1)) # Make sure each sample's 10 values add up to 1.
        return q

    def compute_output_shape(self, input_shape):
        assert input_shape and len(input_shape) == 2
        return input_shape[0], self.n_clusters

    def get_config(self):
        config = {'n_clusters': self.n_clusters}
        base_config = super(ClusteringLayer, self).get_config()
        return dict(list(base_config.items()) + list(config.items()))

In [20]:
def autoencoder(dims, act='relu', init='glorot_uniform'):
    """
    Fully connected auto-encoder model, symmetric.
    Arguments:
        dims: list of number of units in each layer of encoder. dims[0] is input dim, dims[-1] is units in hidden layer.
            The decoder is symmetric with encoder. So number of layers of the auto-encoder is 2*len(dims)-1
        act: activation, not applied to Input, Hidden and Output layers
    return:
        (ae_model, encoder_model), Model of autoencoder and model of encoder
    """
    n_stacks = len(dims) - 1
    # input
    input_img = Input(shape=(dims[0],), name='input')
    x = input_img
    # internal layers in encoder
    for i in range(n_stacks-1):
        x = Dense(dims[i + 1], activation=act, kernel_initializer=init, name='encoder_%d' % i)(x)

    # hidden layer
    encoded = Dense(dims[-1], kernel_initializer=init, name='encoder_%d' % (n_stacks - 1))(x)  # hidden layer, features are extracted from here

    x = encoded
    # internal layers in decoder
    for i in range(n_stacks-1, 0, -1):
        x = Dense(dims[i], activation=act, kernel_initializer=init, name='decoder_%d' % i)(x)

    # output
    x = Dense(dims[0], kernel_initializer=init, name='decoder_0')(x)
    decoded = x
    return Model(inputs=input_img, outputs=decoded, name='AE'), Model(inputs=input_img, outputs=encoded, name='encoder')

In [21]:
# computing an auxiliary target distribution
def target_distribution(q):
    weight = q ** 2 / q.sum(0)
    return (weight.T / weight.sum(1)).T

## Fully Connected DEC

In [22]:
from keras.initializers import VarianceScaling
from keras.optimizers import SGD, Adam

In [23]:
max_count = max([np.max(t) for t in tfidf_data]) * 1.
x = np.divide(tfidf_data, max_count)

# max_count = max([np.max(t) for t in tsne_data]) * 1.
# x = np.divide(tsne_data, max_count)
n_clusters = 8

In [24]:
dims = [x.shape[-1], 500, 500, 2000, 8]
init = VarianceScaling(scale=1. / 3., mode='fan_in', distribution='uniform')
# pretrain_optimizer = SGD(lr=1, momentum=0.9)
pretrain_optimizer = Adam(lr=0.001, beta_1=0.9, beta_2=0.999, epsilon=1e-08, decay=0.0)
pretrain_epochs = 300
batch_size = 256

In [25]:
autoencoder, encoder = autoencoder(dims, init=init)
clustering_layer = ClusteringLayer(n_clusters, name='clustering')(encoder.output)
model = Model(inputs=encoder.input, outputs=[clustering_layer, autoencoder.output])

Instructions for updating:
keep_dims is deprecated, use keepdims instead


In [26]:
model.summary()

____________________________________________________________________________________________________
Layer (type)                     Output Shape          Param #     Connected to                     
input (InputLayer)               (None, 147)           0                                            
____________________________________________________________________________________________________
encoder_0 (Dense)                (None, 500)           74000       input[0][0]                      
____________________________________________________________________________________________________
encoder_1 (Dense)                (None, 500)           250500      encoder_0[0][0]                  
____________________________________________________________________________________________________
encoder_2 (Dense)                (None, 2000)          1002000     encoder_1[0][0]                  
___________________________________________________________________________________________

In [27]:
autoencoder.compile(optimizer=pretrain_optimizer, loss='mse')
%time autoencoder.fit(x, x, batch_size=batch_size, epochs=pretrain_epochs) #, callbacks=cb)

Instructions for updating:
keep_dims is deprecated, use keepdims instead
Epoch 1/300
27960/27960 [==============================] - 5s - loss: 0.0054     
Epoch 2/300
27960/27960 [==============================] - 5s - loss: 0.0036     
Epoch 3/300
27960/27960 [==============================] - 5s - loss: 0.0030     
Epoch 4/300
27960/27960 [==============================] - 5s - loss: 0.0027     
Epoch 5/300
27960/27960 [==============================] - 5s - loss: 0.0024     
Epoch 6/300
27960/27960 [==============================] - 5s - loss: 0.0023     
Epoch 7/300
27960/27960 [==============================] - 5s - loss: 0.0021     
Epoch 8/300
27960/27960 [==============================] - 5s - loss: 0.0020     
Epoch 9/300
27960/27960 [==============================] - 5s - loss: 0.0020     
Epoch 10/300
27960/27960 [==============================] - 5s - loss: 0.0019     
Epoch 11/300
27960/27960 [==============================] - 5s - loss: 0.0018     
Epoch 12/300
27960/2796

27960/27960 [==============================] - 5s - loss: 4.7869e-04     
Epoch 94/300
27960/27960 [==============================] - 5s - loss: 4.7360e-04     
Epoch 95/300
27960/27960 [==============================] - 4s - loss: 4.7666e-04     
Epoch 96/300
27960/27960 [==============================] - 5s - loss: 4.7251e-04     
Epoch 97/300
27960/27960 [==============================] - 4s - loss: 4.5970e-04     
Epoch 98/300
27960/27960 [==============================] - 4s - loss: 4.5608e-04     
Epoch 99/300
27960/27960 [==============================] - 4s - loss: 4.4827e-04     
Epoch 100/300
27960/27960 [==============================] - 4s - loss: 4.4372e-04     
Epoch 101/300
27960/27960 [==============================] - 4s - loss: 4.4822e-04     
Epoch 102/300
27960/27960 [==============================] - 4s - loss: 4.3997e-04     
Epoch 103/300
27960/27960 [==============================] - 5s - loss: 4.3344e-04     
Epoch 104/300
27960/27960 [=========================

27960/27960 [==============================] - 4s - loss: 3.1190e-04     
Epoch 187/300
27960/27960 [==============================] - 4s - loss: 3.1003e-04     
Epoch 188/300
27960/27960 [==============================] - 4s - loss: 3.0933e-04     
Epoch 189/300
27960/27960 [==============================] - 4s - loss: 3.0545e-04     
Epoch 190/300
27960/27960 [==============================] - 4s - loss: 2.9994e-04     
Epoch 191/300
27960/27960 [==============================] - 4s - loss: 2.9885e-04     
Epoch 192/300
27960/27960 [==============================] - 5s - loss: 2.9885e-04     
Epoch 193/300
27960/27960 [==============================] - 5s - loss: 2.9904e-04     
Epoch 194/300
27960/27960 [==============================] - 4s - loss: 2.9756e-04     
Epoch 195/300
27960/27960 [==============================] - 5s - loss: 3.0136e-04     
Epoch 196/300
27960/27960 [==============================] - 4s - loss: 3.0093e-04     
Epoch 197/300
27960/27960 [===================

27960/27960 [==============================] - 5s - loss: 2.6954e-04     
Epoch 280/300
27960/27960 [==============================] - 5s - loss: 2.7077e-04     
Epoch 281/300
27960/27960 [==============================] - 5s - loss: 2.7154e-04     
Epoch 282/300
27960/27960 [==============================] - 5s - loss: 2.7091e-04     
Epoch 283/300
27960/27960 [==============================] - 5s - loss: 2.7250e-04     
Epoch 284/300
27960/27960 [==============================] - 5s - loss: 2.7113e-04     
Epoch 285/300
27960/27960 [==============================] - 5s - loss: 2.7044e-04     
Epoch 286/300
27960/27960 [==============================] - 5s - loss: 2.8022e-04     
Epoch 287/300
27960/27960 [==============================] - 5s - loss: 2.8301e-04     
Epoch 288/300
27960/27960 [==============================] - 5s - loss: 2.8639e-04     
Epoch 289/300
27960/27960 [==============================] - 5s - loss: 2.8818e-04     
Epoch 290/300
27960/27960 [===================

OSError: Unable to create file (unable to open file: name = './data_output/drug_ae_weights_tfidf.h5', errno = 2, error message = 'No such file or directory', flags = 13, o_flags = 602)

In [28]:
autoencoder.save_weights('./model/drug_ae_weights_tfidf_2.h5')
# autoencoder.load_weights('./model/drug_ae_weights_tfidf.h5')

### K-means

In [29]:
kmeans = KMeans(n_clusters=n_clusters, n_init=20)
y_pred = kmeans.fit_predict(encoder.predict(x))
model.get_layer(name='clustering').set_weights([kmeans.cluster_centers_])
y_pred_last = np.copy(y_pred)

## Deep Clustering Train

In [30]:
loss = 0
index = 0
maxiter = 8000
update_interval = 140
index_array = np.arange(x.shape[0])

tol = 0.001 # tolerance threshold to stop training

In [31]:
model.compile(loss=['kld', 'mse'], loss_weights=[0.1, 1], optimizer=pretrain_optimizer)

In [32]:
y = None

In [33]:
for ite in range(int(maxiter)):
    if ite % update_interval == 0:
        q, _  = model.predict(x, verbose=0)
        p = target_distribution(q)  # update the auxiliary target distribution p

        # evaluate the clustering performance
        y_pred = q.argmax(1)
        if y is not None:
            acc = np.round(metrics.accuracy_score(y, y_pred), 5)
            nmi = np.round(metrics.normalized_mutual_info_score(y, y_pred), 5)
            ari = np.round(metrics.adjusted_rand_score(y, y_pred), 5)
            loss = np.round(loss, 5)
            print('Iter %d: acc = %.5f, nmi = %.5f, ari = %.5f' % (ite, acc, nmi, ari), ' ; loss=', loss)

        # check stop criterion
        delta_label = np.sum(y_pred != y_pred_last).astype(np.float32) / y_pred.shape[0]
        y_pred_last = np.copy(y_pred)
        if ite > 0 and delta_label < tol:
            print('delta_label ', delta_label, '< tol ', tol)
            print('Reached tolerance threshold. Stopping training.')
            break
    idx = index_array[index * batch_size: min((index+1) * batch_size, x.shape[0])]
    loss = model.train_on_batch(x=x[idx], y=[p[idx], x[idx]])
    index = index + 1 if (index + 1) * batch_size <= x.shape[0] else 0

delta_label  0.000357653791130186 < tol  0.001
Reached tolerance threshold. Stopping training.


In [34]:
model.save_weights('./model/drug_DEC_model_final_tfidf_2.h5')
# model.load_weights('./model/drug_DEC_model_final_tfidf.h5')

In [35]:
drug_data.head()

,id,title,processed
0,4106,Analysis of efficacy,"[analys, eff]"
1,4107,Comparisons of postoperative CA19-9 levels on survival of ESPAC-4 with the CONOKO-01 and JASPAC-1 trials,"[comparison, postop, ca, level, surv, espac, conoko, jaspac, tri]"
2,4108,Pattern of disease relapse,"[pattern, diseas, relaps]"
3,4109,Grade 1–5 adverse events with gemcitabine alone and gemcitabine plus capecitabine,"[grad, advers, ev, gemcitabin, gemcitabin, plu, capecitabin]"
4,4112,Treatment with zoledronic acid,"[tre, zoledron, acid]"


In [36]:
drug_data = drug_data[['id', 'title']]
drug_data.columns = ['id', 'title']
drug_data.head()

,id,title
0,4106,Analysis of efficacy
1,4107,Comparisons of postoperative CA19-9 levels on survival of ESPAC-4 with the CONOKO-01 and JASPAC-1 trials
2,4108,Pattern of disease relapse
3,4109,Grade 1–5 adverse events with gemcitabine alone and gemcitabine plus capecitabine
4,4112,Treatment with zoledronic acid


In [37]:
drug_data['topic'] = y_pred
drug_data.head()

,id,title,topic
0,4106,Analysis of efficacy,3
1,4107,Comparisons of postoperative CA19-9 levels on survival of ESPAC-4 with the CONOKO-01 and JASPAC-1 trials,2
2,4108,Pattern of disease relapse,7
3,4109,Grade 1–5 adverse events with gemcitabine alone and gemcitabine plus capecitabine,4
4,4112,Treatment with zoledronic acid,4


In [41]:
drug_data.groupby(drug_data.topic).count()

,id,title
topic,,
0,3825,3825
1,3301,3301
2,3873,3873
3,3281,3281
4,3630,3630
5,3671,3671
6,3244,3244
7,3135,3135


In [38]:
drug_data.to_csv('./dec_result.tsv', sep='\t')